In [140]:
#Import all the libraries we need
import os
import json
import pandas as pd

#Get a list of all the folders. The name of the folder is also the id number for the data package. Each folder
#contains the text of the pdf as a txt file and the json result from GNRD.
packages = []
annotation_file = open('dryad_results.txt', 'r')
next(annotation_file)
for line in annotation_file:
    row = line.split('\t')
    data_package = row[1]
    if data_package in packages:
        pass
    else:
        packages.append(data_package)
#packages = next(os.walk('pdf_text'))[1]
packages.remove('133')
print(len(packages))
'110' in packages

219


True

In [141]:
#Iterate over all the folders and grab the json results from each. From the json pick out the name that
#GNRD found.
gnrd_results = {}
tot_words = {}
for p in packages:
    gnrd_names = []
    if p != '133':
        [json_file] = [x for x in os.listdir('dryad-paper/' + p) if x.endswith(".json")]
        [csv_file] = [x for x in os.listdir('dryad-paper/' + p) if x.endswith(".csv")]
        jdata = open('dryad-paper/' + p + '/' + json_file,'r')
        result = json.load(jdata)
        metadata = result['metadata']
        total_words = metadata['totalWords']
        cdata = open('dryad-paper/' + p + '/' + csv_file,'r')
        names = []
        for line in cdata:
            row = line.split(',')
            names.append(row[2])
        gnrd_results[p] = names
        tot_words[p] = total_words
#print(gnrd_results['350'])

In [142]:
#Initialize the dictionary for the human annotator data
annotator_results = {}
for p in packages:
    annotator_results[p]=[]

In [143]:
#Read the file that contains the names found by the human annotators. Go over every row of the file and create a
#dictionary using the data package ID number as the key and a list of the name strings as the value.
# the name string is in a tuple with the type of string, i.e. scientific or vernacular name
annotation_file.seek(0)
next(annotation_file)
for line in annotation_file:
    row = line.split('\t')
    string = row[0]
    data_package = row[1]
    source_type = row[2]
    source = row[3]
    string_type = row[6]
    file_type = row[7]#this is wrong for the publication files
    if source_type == 'data' or source == 'GNRD_1':
        continue
    else:
        try:
            ls = annotator_results[data_package]
        except KeyError:
            print('possible missing data package')
            print(row)
        ls.append(tuple([string,string_type]))
        annotator_results[data_package] = ls
print('completed successfully')
#print(annotator_results['350'])

completed successfully


In [144]:
#Let's compare the list of names returned by GNRD with the list of names returned by the annotators
#for now, let's ignore the vernacular names in the annotator data

error_file = open('errors.txt','a')
error_file.write('data_package' + '\t' + 'name' + '\t' + 'error_type' + '\n')

df = pd.DataFrame(columns=['data_package','true_positives','returned_results','annotator_results','precision',
                           'recall','F1','total_words'])

for p in packages:
    #grab the list for the data package from the gnrd results and the annotator results.
    #separate the name strings from the odds value in the gnrd list and the string type in the annotator list
    gnrd = gnrd_results[p] #name string and odds value
    human = annotator_results[p] #name string and string type
    gnrd_set = set(gnrd)
    human_string = []
    for n,s in human:
        if s != 'vernacular': #lets ignore vernacular names for the moment
            if n.isupper() == True:
                n = n.title()
            human_string.append(n)
    human_set = set(human_string)
    #now we have a set of name strings for gnrd and a set for the annotators
    #let's compare them
    g = len(gnrd_set) #total strings returned by GNRD
    h = len(human_set) #total strings returned by annotator
    overlap_set = gnrd_set.intersection(human_set) #this is the set of strings that are in the gnrd and annotator lists
    o = len(overlap_set)
    gnrd_only = gnrd_set.difference(human_set) #what is in gnrd list that is not in annotator list - false positives
    human_only = human_set.difference(gnrd_set) #what is in annotator list that is not in gnrd list - false negatives
    for x in gnrd_only:
        error_file.write(p + '\t' + x + '\t' + 'false positive' + '\n')
    for y in human_only:
        error_file.write(p + '\t' + y + '\t' + 'false negative' + '\n')
    go = len(gnrd_only)
    ho = len(human_only)
    if go + o == g and ho + o == h: #adding a sanity check
        pass
    else:
        print('problem')
    if g == 0 or h == 0:
        precision = 'NULL'
        recall = 'NULL'
        F1 = 'NULL'
    else:
        precision = o / g
        recall = o / h
        if precision == 0 and recall == 0:
            F1 = 'NULL'
        else:
            F1 = 2*((precision*recall)/(precision+recall))
    #print(df)
    y = {'data_package':p,'true_positives':o,'returned_results':g,'annotator_results':h,
                    'precision':precision,'recall':recall,'F1':F1,'total_words':tot_words[p]}
    df = df.append(y,ignore_index=True)
df.to_csv('results.tsv', sep = '\t')
print(df)

    data_package true_positives returned_results annotator_results precision  \
0              2            104              109               106  0.954128   
1              6              0                1                 0      NULL   
2             12              0                2                 0      NULL   
3             18              5                6                52  0.833333   
4             22            147              150               151      0.98   
..           ...            ...              ...               ...       ...   
214         1480             48               50                52      0.96   
215         1502              5                6                 5  0.833333   
216         1508              8               10                 8       0.8   
217         1519              3                6                 3       0.5   
218         1523              0               16                 0      NULL   

       recall        F1 total_words  
0

In [145]:
#Now we have a dataframe that contains performance data for each pdf 
#let's calculate performance overall
TOT = df.sum(axis=0)
TP = TOT.iloc[1]
RR = TOT.iloc[2]
AR = TOT.iloc[3]
TW = TOT.iloc[4]
print('True Positives ' + str(TP))
print('Returned Results ' + str(RR))
print('Annotator Results ' + str(AR))
print('Total Words ' + str(TW))

True Positives 7952
Returned Results 8929
Annotator Results 9753
Total Words 1589065


In [146]:
precision = TP / RR
recall = TP / AR
F1 = 2*((precision*recall)/(precision+recall))
print(F1)
print('precision is ' + str(precision))
print('recall is ' + str(recall))

0.8513007172679585
precision is 0.8905812520998992
recall is 0.8153388700912539
